# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

In [2]:
# import libraries
import pandas as pd 
import sqlalchemy 
import datetime 
from sqlalchemy import create_engine
import numpy as np
import re

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [6]:
# your code here
# import data set from Ironhack's database
driver = "mysql+pymysql"
user = "ironhacker_read"
password = "ir0nhack3r"
ip = "35.239.232.23"
database = "orders"

connection_string = f'{driver}://{user}:{password}@{ip}/{database}'

engine = create_engine(connection_string)
query = """
        SELECT * FROM orders
"""

orders = pd.read_sql(query, engine)

In [4]:
orders.to_csv("orders.csv", sep=",")

In [12]:
# Needed this step at home as I couldnt connect to the cloud
# orders = pd.read_csv("orders.csv")

In [16]:
orders = orders.drop(columns=["Unnamed: 0", "index"], axis=1)

In [17]:
orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
5,536365,22752,2010,12,3,8,set 7 babushka nesting boxes,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,15.30
6,536365,21730,2010,12,3,8,glass star frosted t-light holder,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,25.50
7,536366,22633,2010,12,3,8,hand warmer union jack,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
8,536366,22632,2010,12,3,8,hand warmer red polka dot,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
9,536367,84879,2010,12,3,8,assorted colour bird ornament,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,54.08


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [19]:
orders.head(1)

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.3


In [108]:
# Sub Problem 1
Total_Spending = orders[["CustomerID", "amount_spent"]].groupby("CustomerID").sum()[["amount_spent"]]
Total_Spending.head(1)

,amount_spent
CustomerID,
12346,77183.6


In [112]:
# Sub Problem 2 & 3
Total_Spending["VIPS"]=Total_Spending["amount_spent"]>(Total_Spending.quantile(q=0.95)["amount_spent"])
Total_Spending.head(1)

,amount_spent,VIPS
CustomerID,,
12346,77183.6,True


In [153]:
Total_Spending["Pref_C"]=(Total_Spending["amount_spent"]>Total_Spending["amount_spent"].quantile(q=0.75)) & (Total_Spending["amount_spent"]<=(Total_Spending["amount_spent"].quantile(q=0.95)))
Total_Spending.head(2)

,amount_spent,VIPS,Pref_C
CustomerID,,,
12346,77183.6,True,False
12347,4310.0,False,True


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [182]:
new_df =pd.merge(orders, Total_Spending, left_on="CustomerID", right_index=True)
new_df.head(2)

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,Total_Spending,amount_spent_y,VIPS,Pref_C
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,NaN,5391.21,False,True
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN,5391.21,False,True


In [181]:
new_df["Country"].describe()

count             397924
unique                37
top       United Kingdom
freq              354345
Name: Country, dtype: object

In [226]:
Country_VIP= new_df[new_df["VIPS"]==True].groupby("Country")["VIPS"].count()

In [227]:
Country_VIP= pd.DataFrame(Country_VIP)

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [228]:
# your code here
Country_Pref= new_df[new_df["Pref_C"]==True].groupby("Country")["Pref_C"].count()

In [229]:
Country_Pref= pd.DataFrame(Country_VIP)

In [230]:
VIP_Pref = Country_VIP + Country_Pref

In [231]:
VIP_Pref

,VIPS
Country,
Australia,1796
Belgium,108
Channel Islands,728
Cyprus,496
Denmark,72
EIRE,14154
Finland,588
France,6580
Germany,6254
